In [1]:
#
# Calibrate StrepA ABM with LFIRE using simulation-step indices 

import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")  # headless

import elfi
import pylfire
from numpy.random import default_rng, SeedSequence
from numpy.random import Generator as _NpGen, RandomState as _RS

from pylfire.classifiers.classifier import Classifier
from sklearn.linear_model import LogisticRegressionCV

import functions_list

In [2]:
# 0) Fixed hyperparameters (order MUST match functions_list.parameters())
# parameters(): [DurationSimulation, Nstrains, Dimmunity, sigma, omega, x,
#                Cperweek, Nagents, alpha, AgeDeath, BasicReproductionNumber]
# ----------------------------
DurationSimulation = 20.0       # years
Nstrains = 42
Dimmunity = 0.5 * 52.14         # weeks
omega = 0.1
x = 10.0
Cperweek = 34.53
Nagents = 2500
alpha = 3.0
AgeDeath = 71.0


# R0_LOW,  R0_HIGH  = 0.0, 5.0  # reproductionNumber 
# SIG_LOW, SIG_HIGH = 0.5, 1.0  # sigma 

In [3]:
# ----------------------------
# 3) Build ABM param vector (theta = [R0, sigma])
# ----------------------------
def build_params(theta2):
    th = np.asarray(theta2, float).ravel()
    if th.size < 2:
        raise ValueError(f"theta2 must be length-2, got {np.shape(theta2)}")
    R0, sigma = float(th[0]), float(th[1])
    return np.array([
        DurationSimulation, Nstrains, Dimmunity, sigma, omega, x,
        Cperweek, Nagents, alpha, AgeDeath, R0
    ], dtype=float)

In [4]:
# ----------------------------
# 4) One ABM run → full prevalence series (length T)
# ----------------------------

def seed_from_theta(theta2, master_seed: int = 123) -> int:
    th = np.asarray(theta2, np.float64).ravel()
    b  = th.tobytes() + np.uint64(master_seed).tobytes()
    return int.from_bytes(hashlib.sha1(b).digest()[:8], 'little')
    
def simulate_prevalence_v5_numba(theta2, seed):
    # derive a child seed from ELFI's rng
    # rng = default_rng(seed)0
    seed = seed_from_theta(theta2, master_seed=seed)
    rng = default_rng(seed)
    params = build_params(theta2)
    AC, IMM, _ = functions_list.initialise_agents_v5(params, rng=rng)

    # call the reproducible simulator that uses only this seed
    SSPrev_selected, SSPrev, AIBKS = functions_list.simulator_v5_numba(
        AC, IMM, params, 0, 1, seed=seed
    )

    # Option A: return the 42x23 matrix (strain × selected times)
    return SSPrev_selected.astype(float)

In [7]:
# ----------------------------
# 7) Robust ELFI random_state → int seed
# ----------------------------
def _seed_from_random_state(random_state):
    if isinstance(random_state, _NpGen):
        return int(random_state.integers(0, 2**32 - 1))
    if isinstance(random_state, _RS):
        return int(random_state.randint(0, 2**32 - 1))
    if isinstance(random_state, (int, np.integer)):
        return int(random_state)
    return int(np.random.randint(0, 2**32 - 1))

In [5]:
# ----------------------------
# 8) ELFI simulator: return simulated values at YOUR indices (idx_valid)
# ----------------------------

def elfi_simulator_v5_numba(R0, sigma, batch_size=1, random_state=None):
    R0    = np.atleast_1d(np.asarray(R0, float))
    sigma = np.atleast_1d(np.asarray(sigma, float))
    bs = int(max(R0.size, sigma.size, int(batch_size)))
    if R0.size == 1:    R0    = np.repeat(R0, bs)
    if sigma.size == 1: sigma = np.repeat(sigma, bs)

    seed = _seed_from_random_state(random_state)
    ss_parent = SeedSequence(seed)
    children = ss_parent.spawn(bs)

    outs = []
    for i in range(bs):
        theta_i = np.array([R0[i], sigma[i]], float)
        # Convert child SeedSequence -> stable 32-bit integer seed
        seed_i = int(children[i].generate_state(1, dtype=np.uint32)[0])
        prev_i = simulate_prevalence_v5_numba(theta_i, seed=seed_i)  # (42, 23)
        outs.append(prev_i)                  # pick your indices only
    return np.stack(outs, axis=0)                       # (bs, len(idx_valid))

In [9]:
# --- Reproducibility test ---
R0 = np.array([2.0])
sigma = np.array([1.0])
parent = 123456  # fixed ELFI random_state (int form)

A = elfi_simulator_v5_numba(R0, sigma, batch_size=4, random_state=None)
B = elfi_simulator_v5_numba(R0, sigma, batch_size=4, random_state=None)

print("Bitwise equal:", np.array_equal(A, B))
print("Allclose     :", np.allclose(A, B))

Bitwise equal: False
Allclose     : False
